In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import logging
import time
import os

logging.basicConfig(level=logging.WARNING)

def quick_info(tickerlist, interval='1d'):
    folder = 'cache'
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    try:
        with open(tickerlist, 'r') as f:
            stocks = [i.strip() for i in f.readlines()]
            if not stocks:
                logging.error('Empty list.')
                return
    except FileNotFoundError:
        logging.error('Tickerfile not found in working directory.')
        return
    
    df = pd.DataFrame(columns=['1y_performance(%)', 'ps_ttm', 'eps_ttm', 'eps_forward', 'est_eps_growth(%)'])
    
    for stock in stocks:
        if not os.path.exists(f'{folder}/{stock}_info.csv'):
            try:
                ticker = yf.Ticker(stock)
                time.sleep(0.2)
                ps_ttm = float(ticker.info.get('priceToSalesTrailing12Months', np.nan))
                eps_ttm = float(ticker.info.get('trailingEps', np.nan))
                eps_forward = float(ticker.info.get('forwardEps', np.nan))
                name = str(ticker.info.get('longName', stock))
                s = pd.Series(dtype='float', data=[ps_ttm, eps_ttm, eps_forward])
                s.index = ['priceToSalesTrailing12Months', 'trailingEps', 'forwardEps']
                s.name = name
                s.to_csv(f'{folder}/{stock}_info.csv')
            except Exception as e:
                logging.warning(f'{stock}: {e}')
                continue
        else:
            ticker_data = pd.read_csv(f'{folder}/{stock}_info.csv', index_col=0)
            ps_ttm = ticker_data.loc['priceToSalesTrailing12Months'][0]
            eps_ttm = ticker_data.loc['trailingEps'][0]
            eps_forward = ticker_data.loc['forwardEps'][0]
            name = ticker_data.columns[0]
        
        diff = (eps_forward/eps_ttm - 1) * 100 if eps_forward > 0 and eps_ttm > 0 else np.nan
                
        if not os.path.exists(f'{folder}/{stock}_ohlc.csv'):
            try:
                time.sleep(0.2)
                ohlc = yf.download(stock, interval=interval, period='2y', auto_adjust=True, progress=False)
                ohlc.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
                if len(ohlc) < 365:
                    logging.warning(f'{stock}: Insufficient ohlc data (only {len(ohlc)} rows)')
                    continue
                ohlc.to_csv(f'{folder}/{stock}_ohlc.csv')
            except Exception as e:
                logging.warning(f'{stock}: {e}')
                continue
        else:
            ohlc = pd.read_csv(f'{folder}/{stock}_ohlc.csv', index_col=0, parse_dates=True)
        
        if len(ohlc) < 365:
            logging.warning(f'{stock}: Insufficient cached ohlc data (only {len(ohlc)} rows)')
            continue
            
        ytd_performance = (ohlc['Close'].iloc[-1] / ohlc['Close'].iloc[-365] - 1) * 100
        
        df.loc[name, '1y_performance(%)'] = round(ytd_performance, 2)
        df.loc[name, 'ps_ttm'] = round(ps_ttm, 2)
        df.loc[name, 'eps_ttm'] = eps_ttm
        df.loc[name, 'eps_forward'] = eps_forward
        df.loc[name, 'est_eps_growth(%)'] = round(diff, 2)
        
    return df.sort_values(by='est_eps_growth(%)', ascending=False)

quick_info('dow-jones.txt')